## SQL basics with Python and SQLite 

#### using a local database "northwind.sqlite" - for a fictional company

https://github.com/jpwhite3/northwind-SQLite3

#### on the KNIME Hub: KNIME, Python and SQL with local SQLite database

https://hub.knime.com/-/spaces/-/latest/~JvzzN5leAp1EgLLU/

<img src="KNIME_loves_Python.png" width="600">

by m_lauber (https://hub.knime.com/mlauber71) and J. Thelen

KNIME and H2 database (https://hub.knime.com/search?type=Workflow&tag=H2,SQL&sort=best)

KNIME and SQLite database (https://hub.knime.com/search?type=Workflow&tag=SQLite,SQL&sort=best)

Play around with SQLite and Date and Time functions (https://hub.knime.com/-/spaces/-/latest/~lr2kArL8kWy1rdfw/)

In [1]:
import sqlite3
import pandas as pd

var_data_path = ''

# Establish connection and cursor
conn = sqlite3.connect(var_data_path + 'northwind.sqlite')
cursor = conn.cursor()

### 001 List all Tables in the SQLite database

In [2]:
query = "SELECT name FROM sqlite_master WHERE type='table'"
print(query)

df_list_tables = pd.read_sql_query(query, conn)
df_list_tables.head(20)

SELECT name FROM sqlite_master WHERE type='table'


,name
0,Categories
1,sqlite_sequence
2,CustomerCustomerDemo
3,CustomerDemographics
4,Customers
5,Employees
6,EmployeeTerritories
7,Order Details
8,Orders
9,Products


### 005 Get Information about a specific table (Products)

In [3]:
query = "SELECT * FROM PRAGMA_TABLE_INFO('Products');"
print(query)

df_list_table_products = pd.read_sql_query(query, conn)
df_list_table_products.head(20)

SELECT * FROM PRAGMA_TABLE_INFO('Products');


,cid,name,type,notnull,dflt_value,pk
0,0,ProductID,INTEGER,1,None,1
1,1,ProductName,TEXT,1,None,0
2,2,SupplierID,INTEGER,0,None,0
3,3,CategoryID,INTEGER,0,None,0
4,4,QuantityPerUnit,TEXT,0,None,0
5,5,UnitPrice,NUMERIC,0,0,0
6,6,UnitsInStock,INTEGER,0,0,0
7,7,UnitsOnOrder,INTEGER,0,0,0
8,8,ReorderLevel,INTEGER,0,0,0
9,9,Discontinued,TEXT,1,'0',0


### 010 Get all data from the Table Products

In [4]:
query = "SELECT * FROM Products;"
print(query)

df_get_table_products = pd.read_sql_query(query, conn)
df_get_table_products.head()

SELECT * FROM Products;


,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1


### 015 Get the number of CustomerIDs from the Orders table

In [5]:
query = "SELECT CustomerID, COUNT(*) AS num_orders FROM Orders GROUP BY CustomerID ORDER BY num_orders DESC"
print(query)

df_get_no_cust_orders = pd.read_sql_query(query, conn)
df_get_no_cust_orders.head(10)

SELECT CustomerID, COUNT(*) AS num_orders FROM Orders GROUP BY CustomerID ORDER BY num_orders DESC


,CustomerID,num_orders
0,SAVEA,31
1,ERNSH,30
2,QUICK,28
3,HUNGO,19
4,FOLKO,19
5,RATTC,18
6,HILAA,18
7,BERGS,18
8,BONAP,17
9,WARTH,15


#### 020 Write the dataframe back to the SQLite database

In [6]:
df_get_no_cust_orders.to_sql(name='CustomerID_NumOrders', con=conn, if_exists='replace', index=False)

# Commit the changes and close the connection
conn.commit()

In [7]:
query = "SELECT * FROM CustomerID_NumOrders;"
print(query)

df_get_table_cs_num_orders = pd.read_sql_query(query, conn)
df_get_table_cs_num_orders.head()

SELECT * FROM CustomerID_NumOrders;


,CustomerID,num_orders
0,SAVEA,31
1,ERNSH,30
2,QUICK,28
3,HUNGO,19
4,FOLKO,19


### 030 Select all Orders from Customer named "ALFKI"

In [8]:
query = "SELECT * FROM Customers WHERE CustomerID='ALFKI'"
print(query)

df_get_table_customer_alfki = pd.read_sql_query(query, conn)
df_get_table_customer_alfki.head()

SELECT * FROM Customers WHERE CustomerID='ALFKI'


,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,Western Europe,12209,Germany,030-0074321,030-0076545


### 035 Calculate the total revenue for each product

In [9]:
query = "SELECT ProductID, SUM(UnitPrice * Quantity) AS Total_Revenue FROM 'Order Details' GROUP BY ProductID ORDER BY Total_Revenue DESC"
print(query)

df_get_revenue_per_product = pd.read_sql_query(query, conn)
df_get_revenue_per_product.head(15)

SELECT ProductID, SUM(UnitPrice * Quantity) AS Total_Revenue FROM 'Order Details' GROUP BY ProductID ORDER BY Total_Revenue DESC


,ProductID,Total_Revenue
0,38,149984.2
1,29,87736.4
2,59,76296.0
3,60,50286.0
4,62,49827.9
5,56,45121.2
6,51,44742.6
7,17,35482.2
8,18,31987.5
9,28,26865.6


### 040 Classify products as 'Expensive' or 'Cheap' based on their unit price

note how to combine lines with \

In [10]:
query = " SELECT ProductName, UnitPrice, " \
        " CASE WHEN UnitPrice >= 50 THEN 'Expensive' ELSE 'Cheap' END AS PriceClass " \
        " FROM Products"
print(query)

df_get_product_expensive = pd.read_sql_query(query, conn)
df_get_product_expensive.head(15)

 SELECT ProductName, UnitPrice,  CASE WHEN UnitPrice >= 50 THEN 'Expensive' ELSE 'Cheap' END AS PriceClass  FROM Products


,ProductName,UnitPrice,PriceClass
0,Chai,18.00,Cheap
1,Chang,19.00,Cheap
2,Aniseed Syrup,10.00,Cheap
3,Chef Anton's Cajun Seasoning,22.00,Cheap
4,Chef Anton's Gumbo Mix,21.35,Cheap
5,Grandma's Boysenberry Spread,25.00,Cheap
6,Uncle Bob's Organic Dried Pears,30.00,Cheap
7,Northwoods Cranberry Sauce,40.00,Cheap
8,Mishi Kobe Niku,97.00,Expensive
9,Ikura,31.00,Cheap


### 045 Calculate the total revenue for each product, and classify as 'High' or 'Low' based on revenue

In [11]:
query = " SELECT ProductID, SUM(UnitPrice * Quantity) AS Revenue, " \
        " CASE WHEN SUM(UnitPrice * Quantity) >= 7500 THEN 'High' ELSE 'Low' END AS RevenueClass " \
        " FROM 'Order Details' GROUP BY ProductID "
print(query)

df_get_product_revenue = pd.read_sql_query(query, conn)
df_get_product_revenue.head(15)

 SELECT ProductID, SUM(UnitPrice * Quantity) AS Revenue,  CASE WHEN SUM(UnitPrice * Quantity) >= 7500 THEN 'High' ELSE 'Low' END AS RevenueClass  FROM 'Order Details' GROUP BY ProductID 


,ProductID,Revenue,RevenueClass
0,1,14277.60,High
1,2,18559.20,High
2,3,3080.00,Low
3,4,9424.80,High
4,5,5801.15,Low
5,6,7345.00,Low
6,7,22464.00,High
7,8,13760.00,High
8,9,8827.00,High
9,10,22140.20,High


### 050 Execute the SQL query to join the Products table with the revenue and revenue class data

use a placeholder "T1" to identify a table. Also you can write a block ofg SQL using the three quotation marks """

In [12]:
query = """
SELECT Products.ProductID, Products.ProductName, Products.CategoryID, 
    SUM(T1.UnitPrice * T1.Quantity) AS Revenue,
    CASE WHEN SUM(T1.UnitPrice * T1.Quantity) >= 7500 THEN 'High' ELSE 'Low' END AS RevenueClass 
FROM Products 
LEFT JOIN 'Order Details' AS T1 ON Products.ProductID = T1.ProductID 
GROUP BY Products.ProductID
"""

print(query)

df_get_product_revenue_details = pd.read_sql_query(query, conn)
df_get_product_revenue_details.head(15)



SELECT Products.ProductID, Products.ProductName, Products.CategoryID, 
    SUM(T1.UnitPrice * T1.Quantity) AS Revenue,
    CASE WHEN SUM(T1.UnitPrice * T1.Quantity) >= 7500 THEN 'High' ELSE 'Low' END AS RevenueClass 
FROM Products 
LEFT JOIN 'Order Details' AS T1 ON Products.ProductID = T1.ProductID 
GROUP BY Products.ProductID



,ProductID,ProductName,CategoryID,Revenue,RevenueClass
0,1,Chai,1,14277.60,High
1,2,Chang,1,18559.20,High
2,3,Aniseed Syrup,2,3080.00,Low
3,4,Chef Anton's Cajun Seasoning,2,9424.80,High
4,5,Chef Anton's Gumbo Mix,2,5801.15,Low
5,6,Grandma's Boysenberry Spread,2,7345.00,Low
6,7,Uncle Bob's Organic Dried Pears,7,22464.00,High
7,8,Northwoods Cranberry Sauce,2,13760.00,High
8,9,Mishi Kobe Niku,6,8827.00,High
9,10,Ikura,8,22140.20,High


#### 055 export result to Excel

In [13]:
df_get_product_revenue_details.to_excel('Product_Revenue_Details.xlsx', sheet_name='Sheet1', index=False)

### 060 Select Orders with a certain Shipped date

In [14]:
start_date = '2016-08-02'

In [15]:
# Execute the SQL query with the start date variable using f-string formatting
query = f"SELECT * FROM Orders WHERE ShippedDate >= '{start_date}';"
print(query)

df_get_table_orders_shipped = pd.read_sql_query(query, conn)
df_get_table_orders_shipped.head()

SELECT * FROM Orders WHERE ShippedDate >= '2016-08-02';


,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10264,FOLKO,6,2016-07-24,2016-08-21,2016-08-23,3,3.67,Folk och fä HB,Åkergatan 24,Bräcke,Northern Europe,S-844 67,Sweden
1,10265,BLONP,2,2016-07-25,2016-08-22,2016-08-12,1,55.28,Blondel père et fils,"24, place Kléber",Strasbourg,Western Europe,67000,France
2,10267,FRANK,4,2016-07-29,2016-08-26,2016-08-06,1,208.58,Frankenversand,Berliner Platz 43,München,Western Europe,80805,Germany
3,10268,GROSR,8,2016-07-30,2016-08-27,2016-08-02,3,66.29,GROSELLA-Restaurante,5ª Ave. Los Palos Grandes,Caracas,South America,1081,Venezuela
4,10269,WHITC,5,2016-07-31,2016-08-14,2016-08-09,1,4.56,White Clover Markets,1029 - 12th Ave. S.,Seattle,North America,98124,USA


## 070 a more complicated example

using an existing list of customer IDs

In [16]:
list_customer_ids = tuple(df_get_table_orders_shipped['CustomerID'].tolist())

print("These are the first 15 items: {}".format(list_customer_ids[:15]))

These are the first 15 items: ('FOLKO', 'BLONP', 'FRANK', 'GROSR', 'WHITC', 'WARTH', 'SPLIR', 'RATTC', 'QUICK', 'VINET', 'MAGAA', 'TORTU', 'MORGK', 'BERGS', 'LEHMS')


In [17]:
# Execute another SQL query to select CompanyName values from the Customers table for the distinct CustomerID values
# Execute the SQL query with the start date variable
# sequence of used variables by the position of brackets

query = """
SELECT 'has product shipped after {}' AS remark, 
CustomerID, CompanyName FROM Customers WHERE CustomerID IN {}
""".format(start_date, list_customer_ids)

# Fetch the results into another pandas dataframe
df_customers_shipped_from = pd.read_sql_query(query, conn)
df_customers_shipped_from.head(15)

,remark,CustomerID,CompanyName
0,has product shipped after 2016-08-02,ALFKI,Alfreds Futterkiste
1,has product shipped after 2016-08-02,ANATR,Ana Trujillo Emparedados y helados
2,has product shipped after 2016-08-02,ANTON,Antonio Moreno Taquería
3,has product shipped after 2016-08-02,AROUT,Around the Horn
4,has product shipped after 2016-08-02,BERGS,Berglunds snabbköp
5,has product shipped after 2016-08-02,BLAUS,Blauer See Delikatessen
6,has product shipped after 2016-08-02,BLONP,Blondesddsl père et fils
7,has product shipped after 2016-08-02,BOLID,Bólido Comidas preparadas
8,has product shipped after 2016-08-02,BONAP,Bon app'
9,has product shipped after 2016-08-02,BOTTM,Bottom-Dollar Markets


### 080 Get an overview about all Pandas dataframes (df*) created

In [18]:
# import pandas as pd
import sys

# Get a dictionary of global variables
global_vars = globals()

# Filter for variables that are pandas dataframes and start with "df"
df_vars = {key: value for key, value in global_vars.items() if isinstance(value, pd.DataFrame) and key.startswith('df')}

# Create a list of dictionaries containing dataframe details
df_details = []
for name, df in df_vars.items():
    df_details.append({
        'name': name,
        'rows': df.shape[0],
        'columns': df.shape[1],
        'memory_usage': sys.getsizeof(df)
    })

# Convert the list of dictionaries to a pandas dataframe
df_details = pd.DataFrame(df_details)

# Convert the memory_usage column to kilobytes
df_details['memory_usage_kb'] = df_details['memory_usage'] / 1024

# Convert the memory_usage column to megabytes
df_details['memory_usage_mb'] = df_details['memory_usage'] / (1024**2)

# Convert the memory_usage column to gigabytes
df_details['memory_usage_gb'] = df_details['memory_usage'] / (1024**3)

# Print the dataframe
df_details.head(25)


,name,rows,columns,memory_usage,memory_usage_kb,memory_usage_mb,memory_usage_gb
0,df_list_tables,15,1,1183,1.155273,0.001128,1.101755e-06
1,df_list_table_products,10,6,2109,2.059570,0.002011,1.964159e-06
2,df_get_table_products,77,10,20475,19.995117,0.019526,1.906883e-05
3,df_get_no_cust_orders,89,2,6374,6.224609,0.006079,5.936250e-06
4,df_get_table_cs_num_orders,89,2,6374,6.224609,0.006079,5.936250e-06
5,df_get_table_customer_alfki,1,11,894,0.873047,0.000853,8.326024e-07
6,df_get_revenue_per_product,77,2,1376,1.343750,0.001312,1.281500e-06
7,df_get_product_expensive,77,3,11596,11.324219,0.011059,1.079962e-05
8,df_get_product_revenue,77,3,6048,5.906250,0.005768,5.632639e-06
9,df_get_product_revenue_details,77,5,12698,12.400391,0.012110,1.182593e-05


### 090 Execute the VACUUM command to optimize the database file

In [19]:
cursor.execute("VACUUM")

#### 100 in the end close the SQLite connection

In [20]:
conn.close()